In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# change working directory
%cd "/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation"

/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation


In [3]:
# install requirements
!pip install json_tricks
!pip install munkres
!pip install ffmpeg
!pip install tensorboardX

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp37-none-any.whl size=6083 sha256=c9be816170f27c840797a7bebf54fa2c2844c763b07039dfca558e0a26553a69
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
     |████████████████████████████████| 122kB 8.9MB/s 


In [4]:
import argparse
import ast
import os
import random
import sys
from datetime import datetime
import cv2

import numpy as np
import torch

sys.path.insert(1, os.getcwd())
from tools.Train_RealExcavators import Train_RealExcavators
from datasets.RealExcavatorsDataset import RealExcavatorsDataset


from misc.Transforms import Rescale

# '--checkpoint_path', '/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/logs/20200730_0409/checkpoint_last.pth'

# pass in the arguments
sys.argv = ['train_colab',
            '--pretrained_weight_path', '/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/logs/Experiment_1_results/Vis0_FDR_15k_20201006_2323_checkpoint_last.pth', 
            '--epochs', '20',
            '--image_resolution', '(384, 288)',
            '--lr', '0.00001',
            '--batch_size', '8',
            '--train_dataset_dir', '/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/datasets/RealExcavators_Augmented/train',
            '--val_dataset_dir', '/content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/datasets/RealExcavators_Augmented/val'
            ]


def main(exp_name,
         epochs=10,
         batch_size=2,
         num_workers=2,
         lr=0.0001,
         disable_lr_decay=False,
         lr_decay_steps='(170, 200)',
         lr_decay_gamma=0.1,
         optimizer='Adam',
         weight_decay=0.,
         momentum=0.9,
         nesterov=False,
         pretrained_weight_path=None,
         checkpoint_path=None,
         log_path='./logs',
         disable_tensorboard_log=False,
         model_c=48,
         model_nof_joints=17,
         model_bn_momentum=0.1,
         disable_flip_test_images=None,
         image_resolution='(384, 288)',
         coco_root_path="./datasets/COCO/",
         coco_bbox_path=None,
         seed=1,
         device=None,
         train_dataset_dir='not added',
         val_dataset_dir='not added'
         ):

    # Seeds
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True  # Enables cudnn
        torch.backends.cudnn.benchmark = True  # It should improve runtime performances when batch shape is fixed. See https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
        torch.backends.cudnn.deterministic = True  # To have ~deterministic results

    # torch device
    if device is not None:
        device = torch.device(device)
    else:
        if torch.cuda.is_available():
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')

    print(device)

    print("\nStarting experiment `%s` @ %s\n" % (exp_name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

    lr_decay = not disable_lr_decay
    use_tensorboard = not disable_tensorboard_log
    flip_test_images = not disable_flip_test_images
    image_resolution = ast.literal_eval(image_resolution)
    lr_decay_steps = ast.literal_eval(lr_decay_steps)

    print("\nLoading train and validation datasets...")

    # load train and val datasets
    ds_train = RealExcavatorsDataset(dataset_dir = train_dataset_dir,
               is_train = True,
               image_width = 288,
               image_height = 384,
               color_rgb = True,
               heatmap_sigma = 3
               )

    ds_val = RealExcavatorsDataset(dataset_dir = val_dataset_dir,
               is_train = True,
               image_width = 288,
               image_height = 384,
               color_rgb = True,
               heatmap_sigma = 3
               )

    train = Train_RealExcavators(
        exp_name=exp_name,
        ds_train=ds_train,
        ds_val=ds_val,
        epochs=epochs,
        batch_size=batch_size,
        num_workers=num_workers,
        loss='JointsMSELoss',
        lr=lr,
        lr_decay=lr_decay,
        lr_decay_steps=lr_decay_steps,
        lr_decay_gamma=lr_decay_gamma,
        optimizer=optimizer,
        weight_decay=weight_decay,
        momentum=momentum,
        nesterov=nesterov,
        pretrained_weight_path=pretrained_weight_path,
        checkpoint_path=checkpoint_path,
        log_path=log_path,
        use_tensorboard=False,
        model_c=model_c,
        model_nof_joints=model_nof_joints,
        model_bn_momentum=model_bn_momentum,
        flip_test_images=False,
        device=device,
        train_dataset_dir = train_dataset_dir,
        val_dataset_dir = val_dataset_dir
    )

    train.run()


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--exp_name", "-n",
                        help="experiment name. A folder with this name will be created in the log_path.",
                        type=str, default=str(datetime.now().strftime("%Y%m%d_%H%M")))
    parser.add_argument("--epochs", "-e", help="number of epochs", type=int, default=200)
    parser.add_argument("--batch_size", "-b", help="batch size", type=int, default=16)
    parser.add_argument("--num_workers", "-w", help="number of DataLoader workers", type=int, default=4)
    parser.add_argument("--lr", "-l", help="initial learning rate", type=float, default=0.001)
    parser.add_argument("--disable_lr_decay", help="disable learning rate decay", action="store_true")
    parser.add_argument("--lr_decay_steps", help="learning rate decay steps", type=str, default="(170, 200)")
    parser.add_argument("--lr_decay_gamma", help="learning rate decay gamma", type=float, default=0.1)
    parser.add_argument("--optimizer", "-o", help="optimizer name. Currently, only `SGD` and `Adam` are supported.",
                        type=str, default='Adam')
    parser.add_argument("--weight_decay", help="weight decay", type=float, default=0.)
    parser.add_argument("--momentum", "-m", help="momentum", type=float, default=0.9)
    parser.add_argument("--nesterov", help="enable nesterov", action="store_true")
    parser.add_argument("--pretrained_weight_path", "-p",
                        help="pre-trained weight path. Weights will be loaded before training starts.",
                        type=str, default=None)
    parser.add_argument("--checkpoint_path", "-c",
                        help="previous checkpoint path. Checkpoint will be loaded before training starts. It includes "
                             "the model, the optimizer, the epoch, and other parameters.",
                        type=str, default=None)
    parser.add_argument("--log_path", help="log path. tensorboard logs and checkpoints will be saved here.",
                        type=str, default='./logs')
    parser.add_argument("--disable_tensorboard_log", "-u", help="disable tensorboard logging", action="store_true")
    parser.add_argument("--model_c", help="HRNet c parameter", type=int, default=48)
    parser.add_argument("--model_nof_joints", help="HRNet nof_joints parameter", type=int, default=6)
    parser.add_argument("--model_bn_momentum", help="HRNet bn_momentum parameter", type=float, default=0.1)
    parser.add_argument("--disable_flip_test_images", help="disable image flip during evaluation", action="store_true")
    parser.add_argument("--image_resolution", "-r", help="image resolution", type=str, default='(384, 288)')
    parser.add_argument("--seed", "-s", help="seed", type=int, default=1)
    parser.add_argument("--device", "-d", help="device", type=str, default=None)
    
    parser.add_argument("--train_dataset_dir", type=str, default=None)
    parser.add_argument("--val_dataset_dir", type=str, default=None)
    
    args = parser.parse_args()

    main(**args.__dict__)


cuda

Starting experiment `20210511_2236` @ 2021-05-11 22:36:35


Loading train and validation datasets...

 Annotations loaded
labels path:  /content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/datasets/RealExcavators_Augmented/train/labels/labels.csv 


 Annotations loaded
labels path:  /content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/datasets/RealExcavators_Augmented/val/labels/labels.csv 

cuda 


##
Loading pre-trained weights
Load from:  /content/drive/My Drive/Colab Notebooks/My Projects/Excavator_Pose_Estimation/logs/Experiment_1_results/Vis0_FDR_15k_20201006_2323_checkpoint_last.pth
Pre-trained weights loaded.
train dataset length:  5000
val dataset length:  1408

Training started @ 2021-05-11 22:37:28

------------------------------------


Epoch 1 of 20 @ 2021-05-11 22:37:28


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Validating:   0%|          | 0/176 [00:00<?, ?it/s]


Train: Loss 0.346436 - Accuracy 0.022066



Validating: 100%|██████████| 176/176 [01:49<00:00,  1.61it/s]



Validation: Loss 0.084501 - Accuracy 0.024975
best_loss 0.084501 at epoch 1
best_acc 0.024975 at epoch 1
best_mAP 0.000000 at epoch 1


...


------------------------------------


Epoch 20 of 20 @ 2021-05-12 01:21:51


Validating:   0%|          | 0/176 [00:00<?, ?it/s]


Train: Loss 0.001327 - Accuracy 0.352469



Validating: 100%|██████████| 176/176 [00:37<00:00,  4.74it/s]



Validation: Loss 0.001836 - Accuracy 0.479079
best_acc 0.479079 at epoch 20

Training ended @ 2021-05-12 01:30:13


# For Testing



In [5]:
import pandas as pd

dataset_dir = '/content/drive/My Drive/Colab Notebooks/Pose_Estimation/Simple-HRNet/datasets/Excavator'
is_train = True
image_width = 288
image_height = 384
color_rgb = True
heatmap_sigma = 3

annotations_path = '/content/drive/My Drive/Colab Notebooks/Pose_Estimation/Simple-HRNet/datasets/Excavator/train/labels/labels.csv'
annotations = pd.read_csv(annotations_path)

#print(annotations.head())

imgIds = annotations['image_id']

print(imgIds[0])

joints = annotations.loc[annotations['image_id'] == imgIds[0]]

print(joints.shape)

joints = joints[['body_end_x', 'body_end_y',
                 'cab_boom_x', 'cab_boom_y',
                 'boom_arm_x', 'boom_arm_y',
                 'arm_bucket_x', 'arm_bucket_y',
                 'bucket_end_left_x', 'bucket_end_left_y',
                 'bucket_end_right_x', 'bucket_end_right_y']]

print(joints.shape)

0.jpg
(1, 20)
(1, 12)


In [6]:
print(type(joints))
print(type(joints_vis))

def _generate_target(self, joints, joints_vis):
  """
  :param joints: [no_of_joints, 2]
  :param joints_vis: [no_of_joints, 2]
  :return: target, target_weight(1: visible, 0: invisible)
  """
  # initialize target_weight
  target_weight = np.ones((self.no_of_joints, 2), dtype=np.float)
  target_weight[:, 0] = joints_vis[:, 0]

  if self.heatmap_type == 'gaussian':
    # initialize target (target is the gaussian heatmap for each joint)
    target = np.zeros((self.no_of_joints,
                        self.heatmap_size[1],
                        self.heatmap_size[0]),
                      dtype=np.float32)
    
    tmp_size = self.heatmap_sigma * 3   # don't exactly know why

    for joint_id in range(self.no_of_joints):
      feat_stride = np.asarray(self.image_size) / np.asarray(self.heatmap_size)
      mu_x = int(joints[joint_id][0] / feat_stride[0] + 0.5)
      mu_y = int(joints[joint_id][1] / feat_stride[1] + 0.5)
      # Check that any part of the gaussian is in-bounds
      ul = [int(mu_x - tmp_size), int(mu_y - tmp_size)]
      br = [int(mu_x + tmp_size + 1), int(mu_y + tmp_size + 1)]
      if ul[0] >= self.heatmap_size[0] or ul[1] >= self.heatmap_size[1] or br[0] < 0 or br[1] < 0:
        # If not, just return the image as is
        target_weight[joint_id] = 0
        continue
        
      # # Generate gaussian
      size = 2 * tmp_size + 1
      x = np.arange(0, size, 1, np.float32)
      y = x[:, np.newaxis]
      x0 = y0 = size // 2
      # The gaussian is not normalized, we want the center value to equal 1
      g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * self.heatmap_sigma ** 2))

      # Usable gaussian range
      g_x = max(0, -ul[0]), min(br[0], self.heatmap_size[0]) - ul[0]
      g_y = max(0, -ul[1]), min(br[1], self.heatmap_size[1]) - ul[1]
      # Image range
      img_x = max(0, ul[0]), min(br[0], self.heatmap_size[0])
      img_y = max(0, ul[1]), min(br[1], self.heatmap_size[1])

      v = target_weight[joint_id]
      print('\n######\n')
      print(type(v))
      print(v)
      print('\n######\n')
      
      if v > 0.5:
        target[joint_id][img_y[0]:img_y[1], img_x[0]:img_x[1]] = g[g_y[0]:g_y[1], g_x[0]:g_x[1]]
      
  else:
    raise NotImplementedError

  return target, target_weight

<class 'pandas.core.frame.DataFrame'>


NameError: ignored